In [1]:
%matplotlib inline
from __future__ import division, print_function
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import ZeroPadding2D
from keras.layers import MaxPooling2D, \
    Lambda, \
    Dense, \
    Dropout, \
    Flatten
from keras.optimizers import Adam
import numpy as np
from keras.preprocessing import image 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from utils import plots

Using TensorFlow backend.


In [2]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

In [3]:
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1]    

In [4]:
idg = image.ImageDataGenerator()
train_generator = idg.flow_from_directory('/mnt/data/dogscats/sample/', target_size=(224, 224), 
                                          batch_size=4, class_mode='categorical', shuffle=True)

Found 24 images belonging to 2 classes.


In [5]:
def addConvBlock(model, layers, filters):
    for layer in range(0, layers):
        model.add(ZeroPadding2D(padding=(1, 1)))
        model.add(Convolution2D(nb_filter=filters, nb_row=3, nb_col=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

In [6]:
def addFCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [7]:
def create():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3, 224, 224), output_shape=(3,224,224)))
    addConvBlock(model, 2, 64)
    addConvBlock(model, 2, 128)
    addConvBlock(model, 3, 256)
    addConvBlock(model, 3, 512)
    addConvBlock(model, 3, 512)
    model.add(Flatten())
    addFCBlock(model)
    addFCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model

In [8]:
model = create()

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [10]:
def get_batches(path, gen=image.ImageDataGenerator(), batch_size=4, class_mode='categorical', shuffle=True):
    return gen.flow_from_directory(path, target_size=(224, 224), batch_size=batch_size, 
                                   class_mode=class_mode, shuffle=shuffle)

In [11]:
def finetune(model):
    model.pop()
    for layer in model.layers:
        layer.trainable = False
    model.add(Dense(2, activation='softmax'))

In [12]:
finetune(model)

In [13]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit_generator(train_generator, samples_per_epoch=train_generator.nb_sample, nb_epoch=1)

Epoch 1/1
24/24 [==============================] - 4s - loss: 0.6897 - acc: 0.6250     
